In [ ]:
# Data Loading
%pylab inline
import pandas as pd

# global path
train_path = 'F:\\notebook\\ppd\\PPD-Second-Round-Data\\'

# training data
train_master = pd.read_csv(train_path+'Master_train_set.csv',index_col ='Idx',encoding='gb18030')
train_user = pd.read_csv(train_path+'User_train_set.csv',index_col ='Idx',encoding='gb18030')
train_log = pd.read_csv(train_path+'Log_train_set.csv',index_col ='Idx',encoding='gb18030')

print "train_master shape %d x %d" %(train_master.shape[0],train_master.shape[1])
print "train_user shape %d x %d" %(train_user.shape[0],train_user.shape[1])
print "train_log shape %d x %d" %(train_log.shape[0],train_log.shape[1])

# test data
test_master = pd.read_csv(train_path+'Master_test_set.csv',index_col ='Idx',encoding='gb18030')
test_user = pd.read_csv(train_path+'User_test_set.csv',index_col ='Idx',encoding='gb18030')
test_log = pd.read_csv(train_path+'Log_test_set.csv',index_col ='Idx',encoding='gb18030')

print "test_master shape %d x %d" %(test_master.shape[0],test_master.shape[1])
print "test_user shape %d x %d" %(test_user.shape[0],test_user.shape[1])
print "test_log shape %d x %d" %(test_log.shape[0],test_log.shape[1])

# combine training and test data
# fake target variable for append data
test_master['target']=0
test_master['test_ind']=1
train_master['test_ind']=0

# 50-50 split for train and val data
from sklearn import cross_validation
X_train, X_val = cross_validation.train_test_split(train_master , test_size=0.5, random_state=0)
train_master.loc[X_val.index,'test_ind']=2

train_master = train_master.append(test_master)
train_user = train_user.append(test_user)
train_log = train_log.append(test_log)

print "train_master shape %d x %d" %(train_master.shape[0],train_master.shape[1])
print "train_user shape %d x %d" %(train_user.shape[0],train_user.shape[1])
print "train_log shape %d x %d" %(train_log.shape[0],train_log.shape[1])

In [ ]:
# Feature Engineering
import string

#User info update
train_user['UserupdateInfo1'] = train_user.UserupdateInfo1.apply(lambda x:string.upper(x))
train_user['ListingInfo1'] = pd.to_datetime(train_user.ListingInfo1,dayfirst=True)
train_user['UserupdateInfo2'] = pd.to_datetime(train_user.UserupdateInfo2,dayfirst=True)
train_user['UserupdateInfo2_weekday'] = train_user['UserupdateInfo2'].apply(lambda x:x.weekday())
train_user['Daysupdatetolisting'] = train_user['ListingInfo1'] - train_user['UserupdateInfo2']
train_user['Daysupdatetolisting'] = train_user['Daysupdatetolisting'].apply(lambda x: x.astype('timedelta64[D]').astype(int))

#User's overall update(count and recency)
train_part1 = pd.DataFrame(index = train_master.index)
train_user_by = pd.DataFrame(train_user.groupby(level=0)['ListingInfo1'].count())
train_user_by.columns = ['cnt_update']
train_user_by['Daysupdatetolisting'] = train_user.groupby(level=0)['Daysupdatetolisting'].min()
train_user_by['Daysfirsttolisting'] = train_user.groupby(level=0)['Daysupdatetolisting'].max()
train_user_by['Duration_update'] = train_user_by['Daysfirsttolisting'] - train_user_by['Daysupdatetolisting']
train_user_by['Daysupdatestd'] = train_user.groupby(level=0)['Daysupdatetolisting'].std()
train_user_by['cnt_update_listday'] = train_user[train_user['Daysupdatetolisting']==0].groupby(level=0)['Daysupdatetolisting'].count()
df = train_user.join(train_user_by,how='left',rsuffix='_r')
train_user_by['cnt_first_update'] = df[df['Daysupdatetolisting']==df['Daysfirsttolisting']].groupby(level=0)['Daysupdatetolisting'].count()
train_user_by['ratio_cnt_first_update'] = train_user_by['cnt_first_update']/train_user_by['cnt_update']
train_part1 = train_part1.join(train_user_by,how='left')
train_part1.fillna({'cnt_update':0,'Daysupdatetolisting':9999,'Daysfirsttolisting':9999,'cnt_update_listday':0,'Daysupdatestd':0,\
              'Duration_update':0,'cnt_first_update':0,'ratio_cnt_first_update':0},inplace=True)
train_part1['cnt_update_days_listday'] = train_user.groupby(level=0)['Daysupdatetolisting'].apply(lambda x: x.nunique())
train_part1['days_between_two_update'] = train_part1['Duration_update']/train_part1['cnt_update_days_listday']
train_part1['cnt_update_days_listday'].fillna(0, inplace = True)
train_part1['days_between_two_update'].fillna(0, inplace = True)
train_part1['cnt_update_cnt_listday'] = train_user.groupby(level=0)['UserupdateInfo1'].apply(lambda x: x.nunique())
train_part1['cnt_update_cnt_listday'].fillna(0, inplace = True)


#User's update by category
df=train_user.reset_index().set_index(['Idx','UserupdateInfo1']).groupby(level=['Idx','UserupdateInfo1']).count().reset_index()
df['UserupdateInfo1'] = df.UserupdateInfo1.apply(lambda x:'cnt'+x)
df = df.pivot(index='Idx', columns='UserupdateInfo1', values='ListingInfo1')
train_part1 = train_part1.join(df,how='left')
train_part1.fillna(0,inplace=True)

#User's update by recency
df=train_user.reset_index().set_index(['Idx','UserupdateInfo1']).groupby(level=['Idx','UserupdateInfo1']).min().reset_index()
df['UserupdateInfo1'] = df.UserupdateInfo1.apply(lambda x:'dayslast'+x)
df = df.pivot(index='Idx', columns='UserupdateInfo1', values='Daysupdatetolisting')
train_part1 = train_part1.join(df,how='left')
train_part1.fillna(-9999,inplace=True)

#User's update by first update
df=train_user.reset_index().set_index(['Idx','UserupdateInfo1']).groupby(level=['Idx','UserupdateInfo1']).max().reset_index()
df['UserupdateInfo1'] = df.UserupdateInfo1.apply(lambda x:'daysfirst'+x)
df = df.pivot(index='Idx', columns='UserupdateInfo1', values='Daysupdatetolisting')
train_part1 = train_part1.join(df,how='left')
train_part1.fillna(0,inplace=True)

#User's update by weekday
df=train_user.reset_index().set_index(['Idx','UserupdateInfo2_weekday']).groupby(level=['Idx','UserupdateInfo2_weekday']).count().reset_index()
df['UserupdateInfo2_weekday'] = df.UserupdateInfo2_weekday.apply(lambda x:'cnt_weekday'+str(x))
df = df.pivot(index='Idx', columns='UserupdateInfo2_weekday', values='ListingInfo1')
train_part1 = train_part1.join(df,how='left')
train_part1.fillna(0,inplace=True)


#train['cnt_update_listday_0'] = train_user[train_user['Daysupdatetolisting']==0].groupby(level=0)['Daysupdatetolisting'].count()
#train['cnt_update_listday_0'].fillna(0, inplace = True)
train_part1['cnt_update_cnt_listday_0'] = train_user[train_user['Daysupdatetolisting']==0].groupby(level=0)['UserupdateInfo1'].apply(lambda x: x.nunique())
train_part1['cnt_update_cnt_listday_0'].fillna(0, inplace = True)
train_part1['cnt_update_listday_3'] = train_user[train_user['Daysupdatetolisting']<=3].groupby(level=0)['Daysupdatetolisting'].count()
train_part1['cnt_update_listday_3'].fillna(0, inplace = True)
train_part1['cnt_update_days_listday_3'] = train_user[train_user['Daysupdatetolisting']<=3].groupby(level=0)['Daysupdatetolisting'].apply(lambda x: x.nunique())
train_part1['cnt_update_days_listday_3'].fillna(0, inplace = True)
train_part1['cnt_update_cnt_listday_3'] = train_user[train_user['Daysupdatetolisting']<=3].groupby(level=0)['UserupdateInfo1'].apply(lambda x: x.nunique())
train_part1['cnt_update_cnt_listday_3'].fillna(0, inplace = True)
train_part1['cnt_update_listday_7'] = train_user[train_user['Daysupdatetolisting']<=7].groupby(level=0)['Daysupdatetolisting'].count()
train_part1['cnt_update_listday_7'].fillna(0, inplace = True)
train_part1['cnt_update_days_listday_7'] = train_user[train_user['Daysupdatetolisting']<=7].groupby(level=0)['Daysupdatetolisting'].apply(lambda x: x.nunique())
train_part1['cnt_update_days_listday_7'].fillna(0, inplace = True)
train_part1['cnt_update_cnt_listday_7'] = train_user[train_user['Daysupdatetolisting']<=7].groupby(level=0)['UserupdateInfo1'].apply(lambda x: x.nunique())
train_part1['cnt_update_cnt_listday_7'].fillna(0, inplace = True)
train_part1['cnt_update_listday_15'] = train_user[train_user['Daysupdatetolisting']<=15].groupby(level=0)['Daysupdatetolisting'].count()
train_part1['cnt_update_listday_15'].fillna(0, inplace = True)
train_part1['cnt_update_days_listday_15'] = train_user[train_user['Daysupdatetolisting']<=15].groupby(level=0)['Daysupdatetolisting'].apply(lambda x: x.nunique())
train_part1['cnt_update_days_listday_15'].fillna(0, inplace = True)
train_part1['cnt_update_cnt_listday_15'] = train_user[train_user['Daysupdatetolisting']<=15].groupby(level=0)['UserupdateInfo1'].apply(lambda x: x.nunique())
train_part1['cnt_update_cnt_listday_15'].fillna(0, inplace = True)
train_part1['cnt_update_listday_30'] = train_user[train_user['Daysupdatetolisting']<=30].groupby(level=0)['Daysupdatetolisting'].count()
train_part1['cnt_update_listday_30'].fillna(0, inplace = True)
train_part1['cnt_update_days_listday_30'] = train_user[train_user['Daysupdatetolisting']<=30].groupby(level=0)['Daysupdatetolisting'].apply(lambda x: x.nunique())
train_part1['cnt_update_days_listday_30'].fillna(0, inplace = True)
train_part1['cnt_update_cnt_listday_30'] = train_user[train_user['Daysupdatetolisting']<=30].groupby(level=0)['UserupdateInfo1'].apply(lambda x: x.nunique())
train_part1['cnt_update_cnt_listday_30'].fillna(0, inplace = True)

list_tmp = train_user['UserupdateInfo1'].value_counts().index.values.tolist()
for item in list_tmp:
    train_part1['cnt_update'+item+'_listday_0'] = train_user[(train_user['Daysupdatetolisting']==0)&(train_user['UserupdateInfo1']==item)].groupby(level=0)['Daysupdatetolisting'].count()
    train_part1['cnt_update'+item+'_listday_0'].fillna(0, inplace = True)
    train_part1['cnt_update'+item+'_listday_3'] = train_user[(train_user['Daysupdatetolisting']<=3)&(train_user['UserupdateInfo1']==item)].groupby(level=0)['Daysupdatetolisting'].count()
    train_part1['cnt_update'+item+'_listday_3'].fillna(0, inplace = True)
    train_part1['cnt_update'+item+'_listday_7'] = train_user[(train_user['Daysupdatetolisting']<=7)&(train_user['UserupdateInfo1']==item)].groupby(level=0)['Daysupdatetolisting'].count()
    train_part1['cnt_update'+item+'_listday_7'].fillna(0, inplace = True)
    train_part1['cnt_update'+item+'_listday_15'] = train_user[(train_user['Daysupdatetolisting']<=15)&(train_user['UserupdateInfo1']==item)].groupby(level=0)['Daysupdatetolisting'].count()
    train_part1['cnt_update'+item+'_listday_15'].fillna(0, inplace = True)
    train_part1['cnt_update'+item+'_listday_30'] = train_user[(train_user['Daysupdatetolisting']<=30)&(train_user['UserupdateInfo1']==item)].groupby(level=0)['Daysupdatetolisting'].count()
    train_part1['cnt_update'+item+'_listday_30'].fillna(0, inplace = True)
    train_part1['cnt_update'+item+'_listday'] = train_user[train_user['UserupdateInfo1']==item].groupby(level=0)['Daysupdatetolisting'].count()
    train_part1['cnt_update'+item+'_listday'].fillna(0, inplace = True)
    train_part1['ratio_cnt_update'+item+'_listday'] = train_part1['cnt_update'+item+'_listday']/train_part1['cnt_update']
    train_part1['ratio_cnt_update'+item+'_listday'].fillna(0, inplace = True)
    train_part1['cnt_update_days'+item+'_listday'] = train_user[train_user['UserupdateInfo1']==item].groupby(level=0)['Daysupdatetolisting'].apply(lambda x: x.nunique())
    train_part1['cnt_update_days'+item+'_listday'].fillna(0, inplace = True)

train_part1['cnt_update_2moreitem_listday_0'] = train_part1[train_part1[['cnt_update'+item+'_listday_0' for item in list_tmp]]>1][['cnt_update'+item+'_listday_0' for item in list_tmp]].count(axis=1)
train_part1['cnt_update_2moreitem_listday_3'] = train_part1[train_part1[['cnt_update'+item+'_listday_3' for item in list_tmp]]>1][['cnt_update'+item+'_listday_3' for item in list_tmp]].count(axis=1)
train_part1['cnt_update_2moreitem_listday_7'] = train_part1[train_part1[['cnt_update'+item+'_listday_7' for item in list_tmp]]>1][['cnt_update'+item+'_listday_7' for item in list_tmp]].count(axis=1)
train_part1['cnt_update_2moreitem_listday_15'] = train_part1[train_part1[['cnt_update'+item+'_listday_15' for item in list_tmp]]>1][['cnt_update'+item+'_listday_15' for item in list_tmp]].count(axis=1)
train_part1['cnt_update_2moreitem_listday_30'] = train_part1[train_part1[['cnt_update'+item+'_listday_30' for item in list_tmp]]>1][['cnt_update'+item+'_listday_30' for item in list_tmp]].count(axis=1)
train_part1['cnt_update_2moreitem_listday'] = train_part1[train_part1[['cnt_update'+item+'_listday' for item in list_tmp]]>1]\
[['cnt_update'+item+'_listday' for item in list_tmp]].count(axis=1)
train_part1['cnt_update_3moreitem_listday'] = train_part1[train_part1[['cnt_update'+item+'_listday' for item in list_tmp]]>2]\
[['cnt_update'+item+'_listday' for item in list_tmp]].count(axis=1)

In [ ]:
#User log
train_log['Listinginfo1'] = pd.to_datetime(train_log.Listinginfo1,dayfirst=True)
train_log['LogInfo3'] = pd.to_datetime(train_log.LogInfo3,dayfirst=True)
train_log['LogInfo3_weekday'] = train_log['LogInfo3'].apply(lambda x:x.weekday())
train_log['LogInfo4'] = train_log['LogInfo2'].apply(lambda x: str(x)+'_') + train_log['LogInfo1'].apply(lambda x: str(x))
train_log['Dayslogtolisting'] = train_log['Listinginfo1'] - train_log['LogInfo3']
train_log['Dayslogtolisting'] = train_log['Dayslogtolisting'].apply(lambda x: x.astype('timedelta64[D]').astype(int))

#User's overall log(count and recency)
train_part2 = pd.DataFrame(index = train_master.index)

train_log_by = pd.DataFrame(train_log.groupby(level=0)['Listinginfo1'].count())
train_log_by.columns = ['cnt_log']
train_log_by['Dayslogtolisting'] = train_log.groupby(level=0)['Dayslogtolisting'].min()
train_log_by['Daysfirstlogtolisting'] = train_log.groupby(level=0)['Dayslogtolisting'].max()
train_log_by['Duration_log'] = train_log_by['Daysfirstlogtolisting'] - train_log_by['Dayslogtolisting']
train_log_by['Dayslogstd'] = train_log.groupby(level=0)['Dayslogtolisting'].std()
train_log_by['cnt_log_listday'] = train_log[train_log['Dayslogtolisting']==0].groupby(level=0)['Dayslogtolisting'].count()
df = train_log.join(train_log_by,how='left',rsuffix='_r')
train_log_by['cnt_first_log'] = df[df['Dayslogtolisting']==df['Daysfirstlogtolisting']].groupby(level=0)['Dayslogtolisting'].count()
df = train_log.reset_index().groupby(['Idx','Dayslogtolisting']).count()
train_log_by['days_since_most_log'] = df[df['Listinginfo1'] == df.groupby(level=0)['Listinginfo1'].transform(max)].reset_index().groupby('Idx')['Dayslogtolisting'].min()
train_log_by['ratio_cnt_first_log'] = train_log_by['cnt_first_log']/train_log_by['cnt_log']
train_part2 = train_part2.join(train_log_by,how='left')
train_part2.fillna({'cnt_log':0,'Dayslogtolisting':9999,'Daysfirstlogtolisting':9999,'cnt_log_listday':0,'Dayslogstd':0,\
              'Duration_log':0,'cnt_first_log':0,'days_since_most_log':0,'ratio_cnt_first_log':0},inplace=True)
train_part2['cnt_log_days_listday'] = train_log.groupby(level=0)['Dayslogtolisting'].apply(lambda x: x.nunique())
train_part2['days_between_two_log'] = train_part2['Duration_log']/train_part2['cnt_log_days_listday']
train_part2['avg_cnt_log'] = train_part2['cnt_log']/train_part2['cnt_log_days_listday']
train_part2['cnt_log_days_listday'].fillna(0, inplace = True)
train_part2['days_between_two_log'].fillna(0, inplace = True)
train_part2['cnt_log_typecnt_listday'] = train_log.groupby(level=0)['LogInfo2'].apply(lambda x: x.nunique())
train_part2['cnt_log_typecnt_listday'].fillna(0, inplace = True)
train_part2['cnt_log_actcnt_listday'] = train_log.groupby(level=0)['LogInfo1'].apply(lambda x: x.nunique())
train_part2['cnt_log_actcnt_listday'].fillna(0, inplace = True)
train_part2['cnt_log_type_actcnt_listday'] = train_log.groupby(level=0)['LogInfo4'].apply(lambda x: x.nunique())
train_part2['cnt_log_type_actcnt_listday'].fillna(0, inplace = True)



#User's log by type*action
df=train_log.reset_index().set_index(['Idx','LogInfo4']).groupby(level=['Idx','LogInfo4']).count().reset_index()
df['LogInfo4'] = df.LogInfo4.apply(lambda x:'cnt_type_act_'+str(x))
df = df.pivot(index='Idx', columns='LogInfo4', values='Listinginfo1')
train_part2 = train_part2.join(df,how='left')
train_part2.fillna(0,inplace=True)

#User's log by weekday
df=train_log.reset_index().set_index(['Idx','LogInfo3_weekday']).groupby(level=['Idx','LogInfo3_weekday']).count().reset_index()
df['LogInfo3_weekday'] = df.LogInfo3_weekday.apply(lambda x:'cnt_log_weekday'+str(x))
df = df.pivot(index='Idx', columns='LogInfo3_weekday', values='Listinginfo1')
train_part2 = train_part2.join(df,how='left')
train_part2.fillna(0,inplace=True)


#User's log by type*action*recency
df=train_log.reset_index().set_index(['Idx','LogInfo4']).groupby(level=['Idx','LogInfo4']).min().reset_index()
df['LogInfo4'] = df.LogInfo4.apply(lambda x:'dayslast_type_act_'+str(x))
df = df.pivot(index='Idx', columns='LogInfo4', values='Dayslogtolisting')
train_part2 = train_part2.join(df,how='left')
train_part2.fillna(-9999,inplace=True)

#User's log by type*action*recency
df=train_log.reset_index().set_index(['Idx','LogInfo4']).groupby(level=['Idx','LogInfo4']).max().reset_index()
df['LogInfo4'] = df.LogInfo4.apply(lambda x:'daysfirst_type_act_'+str(x))
df = df.pivot(index='Idx', columns='LogInfo4', values='Dayslogtolisting')
train_part2 = train_part2.join(df,how='left')
train_part2.fillna(-9999,inplace=True)

#train['cnt_log_listday_0'] = train_log[train_log['Dayslogtolisting']==0].groupby(level=0)['Dayslogtolisting'].count()
#train['cnt_log_listday_0'].fillna(0, inplace = True)
train_part2['cnt_log_type_actcnt_listday_0'] = train_log[train_log['Dayslogtolisting']==0].groupby(level=0)['LogInfo4'].apply(lambda x: x.nunique())
train_part2['cnt_log_type_actcnt_listday_0'].fillna(0, inplace = True)
train_part2['cnt_log_listday_3'] = train_log[train_log['Dayslogtolisting']<=3].groupby(level=0)['Dayslogtolisting'].count()
train_part2['cnt_log_listday_3'].fillna(0, inplace = True)
train_part2['cnt_log_days_listday_3'] = train_log[train_log['Dayslogtolisting']<=3].groupby(level=0)['Dayslogtolisting'].apply(lambda x: x.nunique())
train_part2['cnt_log_days_listday_3'].fillna(0, inplace = True)
train_part2['cnt_log_type_actcnt_listday_3'] = train_log[train_log['Dayslogtolisting']<=3].groupby(level=0)['LogInfo4'].apply(lambda x: x.nunique())
train_part2['cnt_log_type_actcnt_listday_3'].fillna(0, inplace = True)
train_part2['cnt_log_listday_7'] = train_log[train_log['Dayslogtolisting']<=7].groupby(level=0)['Dayslogtolisting'].count()
train_part2['cnt_log_listday_7'].fillna(0, inplace = True)
train_part2['cnt_log_days_listday_7'] = train_log[train_log['Dayslogtolisting']<=7].groupby(level=0)['Dayslogtolisting'].apply(lambda x: x.nunique())
train_part2['cnt_log_days_listday_7'].fillna(0, inplace = True)
train_part2['cnt_log_type_actcnt_listday_7'] = train_log[train_log['Dayslogtolisting']<=7].groupby(level=0)['LogInfo4'].apply(lambda x: x.nunique())
train_part2['cnt_log_type_actcnt_listday_7'].fillna(0, inplace = True)
train_part2['cnt_log_listday_15'] = train_log[train_log['Dayslogtolisting']<=15].groupby(level=0)['Dayslogtolisting'].count()
train_part2['cnt_log_listday_15'].fillna(0, inplace = True)
train_part2['cnt_log_days_listday_15'] = train_log[train_log['Dayslogtolisting']<=15].groupby(level=0)['Dayslogtolisting'].apply(lambda x: x.nunique())
train_part2['cnt_log_days_listday_15'].fillna(0, inplace = True)
train_part2['cnt_log_type_actcnt_listday_15'] = train_log[train_log['Dayslogtolisting']<=15].groupby(level=0)['LogInfo4'].apply(lambda x: x.nunique())
train_part2['cnt_log_type_actcnt_listday_15'].fillna(0, inplace = True)
train_part2['cnt_log_listday_30'] = train_log[train_log['Dayslogtolisting']<=30].groupby(level=0)['Dayslogtolisting'].count()
train_part2['cnt_log_listday_30'].fillna(0, inplace = True)
train_part2['cnt_log_days_listday_30'] = train_log[train_log['Dayslogtolisting']<=30].groupby(level=0)['Dayslogtolisting'].apply(lambda x: x.nunique())
train_part2['cnt_log_days_listday_30'].fillna(0, inplace = True)
train_part2['cnt_log_type_actcnt_listday_30'] = train_log[train_log['Dayslogtolisting']<=30].groupby(level=0)['LogInfo4'].apply(lambda x: x.nunique())
train_part2['cnt_log_type_actcnt_listday_30'].fillna(0, inplace = True)


list_tmp = train_log['LogInfo4'].value_counts().index.values.tolist()
for item in list_tmp:
    train_part2['cnt_loginfo4'+str(item)+'_listday_0'] = train_log[(train_log['Dayslogtolisting']==0)&(train_log['LogInfo4']==item)].groupby(level=0)['Dayslogtolisting'].count()
    train_part2['cnt_loginfo4'+str(item)+'_listday_0'].fillna(0, inplace = True)
    train_part2['cnt_loginfo4'+str(item)+'_listday_3'] = train_log[(train_log['Dayslogtolisting']<=3)&(train_log['LogInfo4']==item)].groupby(level=0)['Dayslogtolisting'].count()
    train_part2['cnt_loginfo4'+str(item)+'_listday_3'].fillna(0, inplace = True)
    train_part2['cnt_loginfo4'+str(item)+'_listday_7'] = train_log[(train_log['Dayslogtolisting']<=7)&(train_log['LogInfo4']==item)].groupby(level=0)['Dayslogtolisting'].count()
    train_part2['cnt_loginfo4'+str(item)+'_listday_7'].fillna(0, inplace = True)
    train_part2['cnt_loginfo4'+str(item)+'_listday_15'] = train_log[(train_log['Dayslogtolisting']<=15)&(train_log['LogInfo4']==item)].groupby(level=0)['Dayslogtolisting'].count()
    train_part2['cnt_loginfo4'+str(item)+'_listday_15'].fillna(0, inplace = True)
    train_part2['cnt_loginfo4'+str(item)+'_listday_30'] = train_log[(train_log['Dayslogtolisting']<=30)&(train_log['LogInfo4']==item)].groupby(level=0)['Dayslogtolisting'].count()
    train_part2['cnt_loginfo4'+str(item)+'_listday_30'].fillna(0, inplace = True)
    train_part2['cnt_loginfo4'+str(item)+'_listday'] = train_log[train_log['LogInfo4']==item].groupby(level=0)['Dayslogtolisting'].count()
    train_part2['cnt_loginfo4'+str(item)+'_listday'].fillna(0, inplace = True)
    train_part2['ratio_cnt_loginfo4'+str(item)+'_listday'] = train_part2['cnt_loginfo4'+str(item)+'_listday']/train_part2['cnt_log']
    train_part2['ratio_cnt_loginfo4'+str(item)+'_listday'].fillna(0, inplace = True)
    train_part2['cnt_log4_days'+str(item)+'_listday'] = train_log[train_log['LogInfo4']==item].groupby(level=0)['Dayslogtolisting'].apply(lambda x: x.nunique())
    train_part2['cnt_log4_days'+str(item)+'_listday'].fillna(0, inplace = True)

train_part2['cnt_log_2moretype_act_listday_0'] = train_part2[train_part2[['cnt_loginfo4'+str(item)+'_listday_0' for item in list_tmp]]>1][['cnt_loginfo4'+str(item)+'_listday_0' for item in list_tmp]].count(axis=1)
train_part2['cnt_log_2moretype_act_listday_3'] = train_part2[train_part2[['cnt_loginfo4'+str(item)+'_listday_3' for item in list_tmp]]>1][['cnt_loginfo4'+str(item)+'_listday_3' for item in list_tmp]].count(axis=1)
train_part2['cnt_log_2moretype_act_listday_7'] = train_part2[train_part2[['cnt_loginfo4'+str(item)+'_listday_7' for item in list_tmp]]>1][['cnt_loginfo4'+str(item)+'_listday_7' for item in list_tmp]].count(axis=1)
train_part2['cnt_log_2moretype_act_listday_15'] = train_part2[train_part2[['cnt_loginfo4'+str(item)+'_listday_15' for item in list_tmp]]>1][['cnt_loginfo4'+str(item)+'_listday_15' for item in list_tmp]].count(axis=1)
train_part2['cnt_log_2moretype_act_listday_30'] = train_part2[train_part2[['cnt_loginfo4'+str(item)+'_listday_30' for item in list_tmp]]>1][['cnt_loginfo4'+str(item)+'_listday_30' for item in list_tmp]].count(axis=1)
train_part2['cnt_log_2moretype_act_listday'] = train_part2[train_part2[['cnt_loginfo4'+str(item)+'_listday' for item in list_tmp]]>1][['cnt_loginfo4'+str(item)+'_listday' for item in list_tmp]].count(axis=1)
train_part2['cnt_log_5moretype_act_listday_0'] = train_part2[train_part2[['cnt_loginfo4'+str(item)+'_listday_0' for item in list_tmp]]>4][['cnt_loginfo4'+str(item)+'_listday_0' for item in list_tmp]].count(axis=1)
train_part2['cnt_log_5moretype_act_listday_3'] = train_part2[train_part2[['cnt_loginfo4'+str(item)+'_listday_3' for item in list_tmp]]>4][['cnt_loginfo4'+str(item)+'_listday_3' for item in list_tmp]].count(axis=1)
train_part2['cnt_log_5moretype_act_listday_7'] = train_part2[train_part2[['cnt_loginfo4'+str(item)+'_listday_7' for item in list_tmp]]>4][['cnt_loginfo4'+str(item)+'_listday_7' for item in list_tmp]].count(axis=1)
train_part2['cnt_log_5moretype_act_listday_15'] = train_part2[train_part2[['cnt_loginfo4'+str(item)+'_listday_15' for item in list_tmp]]>4][['cnt_loginfo4'+str(item)+'_listday_15' for item in list_tmp]].count(axis=1)
train_part2['cnt_log_5moretype_act_listday_30'] = train_part2[train_part2[['cnt_loginfo4'+str(item)+'_listday_30' for item in list_tmp]]>4][['cnt_loginfo4'+str(item)+'_listday_30' for item in list_tmp]].count(axis=1)
train_part2['cnt_log_5moretype_act_listday'] = train_part2[train_part2[['cnt_loginfo4'+str(item)+'_listday' for item in list_tmp]]>4][['cnt_loginfo4'+str(item)+'_listday' for item in list_tmp]].count(axis=1)
train_part2['cnt_log_10moretype_act_listday_0'] = train_part2[train_part2[['cnt_loginfo4'+str(item)+'_listday_0' for item in list_tmp]]>9][['cnt_loginfo4'+str(item)+'_listday_0' for item in list_tmp]].count(axis=1)
train_part2['cnt_log_10moretype_act_listday_3'] = train_part2[train_part2[['cnt_loginfo4'+str(item)+'_listday_3' for item in list_tmp]]>9][['cnt_loginfo4'+str(item)+'_listday_3' for item in list_tmp]].count(axis=1)
train_part2['cnt_log_10moretype_act_listday_7'] = train_part2[train_part2[['cnt_loginfo4'+str(item)+'_listday_7' for item in list_tmp]]>9][['cnt_loginfo4'+str(item)+'_listday_7' for item in list_tmp]].count(axis=1)
train_part2['cnt_log_10moretype_act_listday_15'] = train_part2[train_part2[['cnt_loginfo4'+str(item)+'_listday_15' for item in list_tmp]]>9][['cnt_loginfo4'+str(item)+'_listday_15' for item in list_tmp]].count(axis=1)
train_part2['cnt_log_10moretype_act_listday_30'] = train_part2[train_part2[['cnt_loginfo4'+str(item)+'_listday_30' for item in list_tmp]]>9][['cnt_loginfo4'+str(item)+'_listday_30' for item in list_tmp]].count(axis=1)
train_part2['cnt_log_10moretype_act_listday'] = train_part2[train_part2[['cnt_loginfo4'+str(item)+'_listday' for item in list_tmp]]>9][['cnt_loginfo4'+str(item)+'_listday' for item in list_tmp]].count(axis=1)

In [ ]:
# Master Data

# aggregation functions
def msn(input):
    length = 0
    for item in input.tolist():
        if item!=-1:
            return length
        else:
            length += 1
            
def msz(input):
    length = 0
    for item in input.tolist():
        if item>0:
            return length
        else:
            length += 1
            
def msx(input):
    length = 0
    max_value = input.max()
    for item in input.tolist():
        if item==max_value:
            return length
        else:
            length += 1

def num(input):
    length = 0
    for item in input.tolist():
        if item>-1:
            length += 1
    return length

def nuz(input):
    length = 0
    for item in input.tolist():
        if item>0:
            length += 1
    return length
# cap and floor

'''train_master[train_master['WeblogInfo_9']>=1]['WeblogInfo_9']=1
train_master[train_master['WeblogInfo_8']>=7]['WeblogInfo_8']=7
train_master[train_master['WeblogInfo_7']>=26]['WeblogInfo_7']=26
train_master[train_master['WeblogInfo_6']>=10]['WeblogInfo_6']=10
train_master[train_master['WeblogInfo_5']>=7]['WeblogInfo_5']=7
train_master[train_master['WeblogInfo_4']>=11]['WeblogInfo_4']=11
train_master[train_master['WeblogInfo_18']>=10]['WeblogInfo_18']=10
train_master[train_master['WeblogInfo_17']>=16]['WeblogInfo_17']=16
train_master[train_master['WeblogInfo_16']>=10]['WeblogInfo_16']=10
train_master[train_master['WeblogInfo_15']>=8]['WeblogInfo_15']=8
train_master[train_master['WeblogInfo_14']>=5]['WeblogInfo_14']=5
train_master[train_master['SocialNetwork_9']>=500]['SocialNetwork_9']=500
train_master[train_master['SocialNetwork_8']>=100]['SocialNetwork_8']=100
train_master[train_master['SocialNetwork_6']>=1]['SocialNetwork_6']=1
train_master[train_master['SocialNetwork_5']>=0]['SocialNetwork_5']=0
train_master[train_master['SocialNetwork_4']>=0]['SocialNetwork_4']=0
train_master[train_master['SocialNetwork_3']>=0]['SocialNetwork_3']=0
train_master[train_master['SocialNetwork_10']>=200]['SocialNetwork_10']=200'''
train_master[train_master['UserInfo_24']!='D']['UserInfo_24']='N'
train_master['WeblogInfo_20'].fillna('N').apply(lambda x: x[0])


train_master['ListingInfo'] = pd.to_datetime(train_master.ListingInfo,dayfirst=True)
train_master['ListingInfo_M'] = train_master['ListingInfo'].values.astype('M8[M]')
train_master['ListingInfo_weekday'] = train_master['ListingInfo'].apply(lambda x:x.weekday())
train_master['ListingInfo_weekno'] = train_master['ListingInfo'].apply(lambda x:100*x.isocalendar()[0]+x.isocalendar()[1])

train_master['less_nation_info2'] = train_master['UserInfo_2'].fillna('none').apply(lambda x:1 if u"自治州" in x else 0)
train_master['less_nation_info4'] = train_master['UserInfo_4'].fillna('none').apply(lambda x:1 if u"自治州" in x else 0)
train_master['less_nation_info8'] = train_master['UserInfo_8'].fillna('none').apply(lambda x:1 if u"自治州" in x else 0)

city_ref = pd.read_csv(train_path+'provinc_city_lookup.csv',encoding='gb18030',names=['city','province','class'])
train_master['UserInfo_2_province'] = train_master.merge(right =city_ref,how='left', left_on='UserInfo_2',right_on='city')['province'].values
train_master['UserInfo_4_province'] = train_master.merge(right =city_ref,how='left', left_on='UserInfo_4',right_on='city')['province'].values
train_master['UserInfo_2_class'] = train_master.merge(right =city_ref,how='left', left_on='UserInfo_2',right_on='city')['class'].values
train_master['UserInfo_4_class'] = train_master.merge(right =city_ref,how='left', left_on='UserInfo_4',right_on='city')['class'].values

train_master['UserInfo_7_unknown'] = train_master['UserInfo_7']==u"不详"
train_master['UserInfo_20_unknown'] = train_master['UserInfo_20']==u"不详"

train_master['UserInfo_2_4_same'] = train_master['UserInfo_2']==train_master['UserInfo_4']
train_master['UserInfo_2_4_province_same'] = train_master['UserInfo_2_province']==train_master['UserInfo_4_province']
train_master['UserInfo_2_8_same'] = (train_master['UserInfo_2']==train_master['UserInfo_8'])|(train_master['UserInfo_2']+u"市"==train_master['UserInfo_8'])
train_master['UserInfo_2_8_province_same'] = (train_master['UserInfo_2_province']==train_master['UserInfo_7'])|(train_master['UserInfo_2_province']+u"省"==train_master['UserInfo_7'])
train_master['UserInfo_2_20_same'] = (train_master['UserInfo_2']==train_master['UserInfo_20'])|(train_master['UserInfo_2']+u"市"==train_master['UserInfo_20'])
train_master['UserInfo_2_20_province_same'] = (train_master['UserInfo_2_province']==train_master['UserInfo_19'])|(train_master['UserInfo_2_province']+u"省"==train_master['UserInfo_19'])

train_master['UserInfo_4_8_same'] = (train_master['UserInfo_4']==train_master['UserInfo_8'])|(train_master['UserInfo_4']+u"市"==train_master['UserInfo_8'])
train_master['UserInfo_4_8_province_same'] = (train_master['UserInfo_4_province']==train_master['UserInfo_7'])|(train_master['UserInfo_4_province']+u"省"==train_master['UserInfo_7'])
train_master['UserInfo_4_20_same'] = (train_master['UserInfo_4']==train_master['UserInfo_20'])|(train_master['UserInfo_4']+u"市"==train_master['UserInfo_20'])
train_master['UserInfo_4_20_province_same'] = (train_master['UserInfo_4_province']==train_master['UserInfo_19'])|(train_master['UserInfo_4_province']+u"省"==train_master['UserInfo_19'])

train_master['UserInfo_8_20_same'] = (train_master['UserInfo_8']==train_master['UserInfo_20'])|(train_master['UserInfo_8']+u"市"==train_master['UserInfo_20'])
train_master['UserInfo_8_20_province_same'] = (train_master['UserInfo_7']==train_master['UserInfo_19'])|(train_master['UserInfo_7']+u"省"==train_master[\
'UserInfo_19'])

stock_data  = pd.read_csv(train_path+'stock_v4.csv')
stock_data['date'] = pd.to_datetime(stock_data.date)
tmp = train_master.merge(right =stock_data,how='left', left_on='ListingInfo',right_on='date')[['price',\
            'future_1m_index','Diff_future_1m_index','Ratio_future_1m_index', 'future_1m_up', 'future_1m_down', 'future_1m_up_2pct','future_1m_down_2pct',\
            'future_3m_index','Diff_future_3m_index','Ratio_future_3m_index', 'future_3m_up', 'future_3m_down', 'future_3m_up_2pct','future_3m_down_2pct',\
            'future_6m_index','Diff_future_6m_index','Ratio_future_6m_index', 'future_6m_up', 'future_6m_down', 'future_6m_up_2pct','future_6m_down_2pct',\
            'future_12m_index','Diff_future_12m_index','Ratio_future_12m_index','future_12m_up', 'future_12m_down', 'future_12m_up_2pct','future_12m_down_2pct',\
            ]].values

train_master['price']=tmp[:,0]

ipo_data  = pd.read_csv(train_path+'ipo_data_v2.csv')
ipo_data['date'] = pd.to_datetime(ipo_data.date)
tmp = train_master.merge(right =ipo_data,how='left', left_on='ListingInfo',right_on='date')[['cnt_ipo',\
            'amt_ipo','prob_ipo','ind_ipo_f7d','ind_ipo_f15d','ind_ipo_f30d','cnt_ipo_f7d','cnt_ipo_f15d',\
            'cnt_ipo_f30d','amt_ipo_f7d','amt_ipo_f15d','amt_ipo_f30d','prob_ipo_f7d','prob_ipo_f15d',\
            'prob_ipo_f30d'\
            ]].values

train_master['cnt_ipo']=tmp[:,0]
train_master['amt_ipo']=tmp[:,1]
train_master['prob_ipo']=tmp[:,2]
train_master['ind_ipo_f7d']=tmp[:,3]
train_master['ind_ipo_f15d']=tmp[:,4]
train_master['ind_ipo_f30d']=tmp[:,5]
train_master['cnt_ipo_f7d']=tmp[:,6]
train_master['cnt_ipo_f15d']=tmp[:,7]
train_master['cnt_ipo_f30d']=tmp[:,8]
train_master['amt_ipo_f7d']=tmp[:,9]
train_master['amt_ipo_f15d']=tmp[:,10]
train_master['amt_ipo_f30d']=tmp[:,11]
train_master['prob_ipo_f7d']=tmp[:,12]
train_master['prob_ipo_f15d']=tmp[:,13]
train_master['prob_ipo_f30d']=tmp[:,14]


for i in range(1,18):
    tmplist = ['ThirdParty_Info_Period'+str(j)+'_'+str(i) for j in range(1,8)]
    train_master['ThirdParty_Info'+'_'+str(i)+'_sum'] = train_master[tmplist].replace({-1:np.nan}).sum(axis=1,skipna=True).fillna(-1)
    train_master['ThirdParty_Info'+'_'+str(i)+'_mean'] = train_master[tmplist].replace({-1:np.nan}).mean(axis=1,skipna=True).fillna(-1)
    train_master['ThirdParty_Info'+'_'+str(i)+'_cnt'] = train_master[tmplist].replace({-1:np.nan}).count(axis=1).fillna(-1)
    train_master['ThirdParty_Info'+'_'+str(i)+'_max'] = train_master[tmplist].replace({-1:np.nan}).max(axis=1,skipna=True).fillna(-1)
    train_master['ThirdParty_Info'+'_'+str(i)+'_min'] = train_master[tmplist].replace({-1:np.nan}).min(axis=1,skipna=True).fillna(-1)
    train_master['ThirdParty_Info'+'_'+str(i)+'_std'] = train_master[tmplist].replace({-1:np.nan}).std(axis=1,skipna=True).fillna(-1)
    train_master['ThirdParty_Info'+'_'+str(i)+'_cmx'] = train_master['ThirdParty_Info_Period1'+'_'+str(i)]/train_master['ThirdParty_Info'+'_'+str(i)+'_max']
    train_master['ThirdParty_Info'+'_'+str(i)+'_cav'] = train_master['ThirdParty_Info_Period1'+'_'+str(i)]/train_master['ThirdParty_Info'+'_'+str(i)+'_mean']
    train_master['ThirdParty_Info'+'_'+str(i)+'_msn'] = train_master[tmplist].apply(func=msn,axis=1).fillna(9999)
    train_master['ThirdParty_Info'+'_'+str(i)+'_msz'] = train_master[tmplist].apply(func=msz,axis=1).fillna(9999)
    train_master['ThirdParty_Info'+'_'+str(i)+'_msx'] = train_master[tmplist].apply(func=msx,axis=1).fillna(-1)
    train_master['ThirdParty_Info'+'_'+str(i)+'_num'] = train_master[tmplist].apply(func=num,axis=1).fillna(-1)
    train_master['ThirdParty_Info'+'_'+str(i)+'_nuz'] = train_master[tmplist].apply(func=nuz,axis=1).fillna(-1)
    train_master['ThirdParty_Info'+'_'+str(i)+'_bup_6'] = train_master[tmplist].replace({-1:np.nan}).\
    apply(lambda x:1 if x.iloc[0]>=x.iloc[1]>=x.iloc[2]>=x.iloc[3]>=x.iloc[4]>=x.iloc[5] else 0)
    train_master['ThirdParty_Info'+'_'+str(i)+'_bud_6'] = train_master[tmplist].replace({-1:np.nan}).\
    apply(lambda x:1 if x.iloc[0]<=x.iloc[1]<=x.iloc[2]>=x.iloc[3]>=x.iloc[4]>=x.iloc[5] else 0)
    train_master['ThirdParty_Info'+'_'+str(i)+'_bup_3'] = train_master[tmplist].replace({-1:np.nan}).\
    apply(lambda x:1 if x.iloc[0]>=x.iloc[1]>=x.iloc[2] else 0)
    train_master['ThirdParty_Info'+'_'+str(i)+'_bud_3'] = train_master[tmplist].replace({-1:np.nan}).\
    apply(lambda x:1 if x.iloc[0]<=x.iloc[1]<=x.iloc[2] else 0)
    train_master = train_master.join(train_master[tmplist].replace({-1:np.nan}).rank(axis=1,method='min',ascending=False).fillna(-1),how='left',rsuffix='Rank_')

'''train_master[train_master['ThirdParty_Info_Period1_1']>=200]['ThirdParty_Info_Period1_1']=200
train_master[train_master['ThirdParty_Info_Period2_1']>=200]['ThirdParty_Info_Period2_1']=200
train_master[train_master['ThirdParty_Info_Period3_1']>=200]['ThirdParty_Info_Period3_1']=200
train_master[train_master['ThirdParty_Info_Period4_1']>=200]['ThirdParty_Info_Period4_1']=200
train_master[train_master['ThirdParty_Info_Period5_1']>=200]['ThirdParty_Info_Period5_1']=200
train_master[train_master['ThirdParty_Info_Period6_1']>=200]['ThirdParty_Info_Period6_1']=200
train_master[train_master['ThirdParty_Info_Period7_1']>=200]['ThirdParty_Info_Period7_1']=200
train_master[train_master['ThirdParty_Info_Period1_2']>=170]['ThirdParty_Info_Period1_2']=170
train_master[train_master['ThirdParty_Info_Period2_2']>=170]['ThirdParty_Info_Period2_2']=170
train_master[train_master['ThirdParty_Info_Period3_2']>=170]['ThirdParty_Info_Period3_2']=170
train_master[train_master['ThirdParty_Info_Period4_2']>=170]['ThirdParty_Info_Period4_2']=170
train_master[train_master['ThirdParty_Info_Period5_2']>=170]['ThirdParty_Info_Period5_2']=170
train_master[train_master['ThirdParty_Info_Period6_2']>=170]['ThirdParty_Info_Period6_2']=170
train_master[train_master['ThirdParty_Info_Period7_2']>=170]['ThirdParty_Info_Period7_2']=170
train_master[train_master['ThirdParty_Info_Period1_3']>=1000]['ThirdParty_Info_Period1_3']=1000
train_master[train_master['ThirdParty_Info_Period2_3']>=1000]['ThirdParty_Info_Period2_3']=1000
train_master[train_master['ThirdParty_Info_Period3_3']>=1000]['ThirdParty_Info_Period3_3']=1000
train_master[train_master['ThirdParty_Info_Period4_3']>=1000]['ThirdParty_Info_Period4_3']=1000
train_master[train_master['ThirdParty_Info_Period5_3']>=1000]['ThirdParty_Info_Period5_3']=1000
train_master[train_master['ThirdParty_Info_Period6_3']>=1000]['ThirdParty_Info_Period6_3']=1000
train_master[train_master['ThirdParty_Info_Period7_3']>=1000]['ThirdParty_Info_Period7_3']=1000
train_master[train_master['ThirdParty_Info_Period1_4']>=1000]['ThirdParty_Info_Period1_4']=1000
train_master[train_master['ThirdParty_Info_Period2_4']>=1000]['ThirdParty_Info_Period2_4']=1000
train_master[train_master['ThirdParty_Info_Period3_4']>=1000]['ThirdParty_Info_Period3_4']=1000
train_master[train_master['ThirdParty_Info_Period4_4']>=1000]['ThirdParty_Info_Period4_4']=1000
train_master[train_master['ThirdParty_Info_Period5_4']>=1000]['ThirdParty_Info_Period5_4']=1000
train_master[train_master['ThirdParty_Info_Period6_4']>=1000]['ThirdParty_Info_Period6_4']=1000
train_master[train_master['ThirdParty_Info_Period7_4']>=1000]['ThirdParty_Info_Period7_4']=1000
train_master[train_master['ThirdParty_Info_Period1_5']>=300]['ThirdParty_Info_Period1_5']=300
train_master[train_master['ThirdParty_Info_Period2_5']>=300]['ThirdParty_Info_Period2_5']=300
train_master[train_master['ThirdParty_Info_Period3_5']>=300]['ThirdParty_Info_Period3_5']=300
train_master[train_master['ThirdParty_Info_Period4_5']>=300]['ThirdParty_Info_Period4_5']=300
train_master[train_master['ThirdParty_Info_Period5_5']>=300]['ThirdParty_Info_Period5_5']=300
train_master[train_master['ThirdParty_Info_Period6_5']>=300]['ThirdParty_Info_Period6_5']=300
train_master[train_master['ThirdParty_Info_Period7_5']>=300]['ThirdParty_Info_Period7_5']=300
train_master[train_master['ThirdParty_Info_Period1_6']>=250]['ThirdParty_Info_Period1_6']=250
train_master[train_master['ThirdParty_Info_Period2_6']>=250]['ThirdParty_Info_Period2_6']=250
train_master[train_master['ThirdParty_Info_Period3_6']>=250]['ThirdParty_Info_Period3_6']=250
train_master[train_master['ThirdParty_Info_Period4_6']>=250]['ThirdParty_Info_Period4_6']=250
train_master[train_master['ThirdParty_Info_Period5_6']>=250]['ThirdParty_Info_Period5_6']=250
train_master[train_master['ThirdParty_Info_Period6_6']>=250]['ThirdParty_Info_Period6_6']=250
train_master[train_master['ThirdParty_Info_Period7_6']>=250]['ThirdParty_Info_Period7_6']=250
train_master[train_master['ThirdParty_Info_Period1_7']>=1400]['ThirdParty_Info_Period1_7']=1400
train_master[train_master['ThirdParty_Info_Period2_7']>=1400]['ThirdParty_Info_Period2_7']=1400
train_master[train_master['ThirdParty_Info_Period3_7']>=1400]['ThirdParty_Info_Period3_7']=1400
train_master[train_master['ThirdParty_Info_Period4_7']>=1400]['ThirdParty_Info_Period4_7']=1400
train_master[train_master['ThirdParty_Info_Period5_7']>=1400]['ThirdParty_Info_Period5_7']=1400
train_master[train_master['ThirdParty_Info_Period6_7']>=1400]['ThirdParty_Info_Period6_7']=1400
train_master[train_master['ThirdParty_Info_Period7_7']>=1400]['ThirdParty_Info_Period7_7']=1400
train_master[train_master['ThirdParty_Info_Period1_8']>=500]['ThirdParty_Info_Period1_8']=500
train_master[train_master['ThirdParty_Info_Period2_8']>=500]['ThirdParty_Info_Period2_8']=500
train_master[train_master['ThirdParty_Info_Period3_8']>=500]['ThirdParty_Info_Period3_8']=500
train_master[train_master['ThirdParty_Info_Period4_8']>=500]['ThirdParty_Info_Period4_8']=500
train_master[train_master['ThirdParty_Info_Period5_8']>=500]['ThirdParty_Info_Period5_8']=500
train_master[train_master['ThirdParty_Info_Period6_8']>=500]['ThirdParty_Info_Period6_8']=500
train_master[train_master['ThirdParty_Info_Period7_8']>=500]['ThirdParty_Info_Period7_8']=500
train_master[train_master['ThirdParty_Info_Period1_9']>=30]['ThirdParty_Info_Period1_9']=30
train_master[train_master['ThirdParty_Info_Period2_9']>=30]['ThirdParty_Info_Period2_9']=30
train_master[train_master['ThirdParty_Info_Period3_9']>=30]['ThirdParty_Info_Period3_9']=30
train_master[train_master['ThirdParty_Info_Period4_9']>=30]['ThirdParty_Info_Period4_9']=30
train_master[train_master['ThirdParty_Info_Period5_9']>=30]['ThirdParty_Info_Period5_9']=30
train_master[train_master['ThirdParty_Info_Period6_9']>=30]['ThirdParty_Info_Period6_9']=30
train_master[train_master['ThirdParty_Info_Period7_9']>=30]['ThirdParty_Info_Period7_9']=30
train_master[train_master['ThirdParty_Info_Period1_10']>=7]['ThirdParty_Info_Period1_10']=7
train_master[train_master['ThirdParty_Info_Period2_10']>=7]['ThirdParty_Info_Period2_10']=7
train_master[train_master['ThirdParty_Info_Period3_10']>=7]['ThirdParty_Info_Period3_10']=7
train_master[train_master['ThirdParty_Info_Period4_10']>=7]['ThirdParty_Info_Period4_10']=7
train_master[train_master['ThirdParty_Info_Period5_10']>=7]['ThirdParty_Info_Period5_10']=7
train_master[train_master['ThirdParty_Info_Period6_10']>=7]['ThirdParty_Info_Period6_10']=7
train_master[train_master['ThirdParty_Info_Period7_10']>=7]['ThirdParty_Info_Period7_10']=7
train_master[train_master['ThirdParty_Info_Period1_11']>=70]['ThirdParty_Info_Period1_11']=70
train_master[train_master['ThirdParty_Info_Period2_11']>=70]['ThirdParty_Info_Period2_11']=70
train_master[train_master['ThirdParty_Info_Period3_11']>=70]['ThirdParty_Info_Period3_11']=70
train_master[train_master['ThirdParty_Info_Period4_11']>=70]['ThirdParty_Info_Period4_11']=70
train_master[train_master['ThirdParty_Info_Period5_11']>=70]['ThirdParty_Info_Period5_11']=70
train_master[train_master['ThirdParty_Info_Period6_11']>=70]['ThirdParty_Info_Period6_11']=70
train_master[train_master['ThirdParty_Info_Period7_11']>=70]['ThirdParty_Info_Period7_11']=70
train_master[train_master['ThirdParty_Info_Period1_12']>=15]['ThirdParty_Info_Period1_12']=15
train_master[train_master['ThirdParty_Info_Period2_12']>=15]['ThirdParty_Info_Period2_12']=15
train_master[train_master['ThirdParty_Info_Period3_12']>=15]['ThirdParty_Info_Period3_12']=15
train_master[train_master['ThirdParty_Info_Period4_12']>=15]['ThirdParty_Info_Period4_12']=15
train_master[train_master['ThirdParty_Info_Period5_12']>=15]['ThirdParty_Info_Period5_12']=15
train_master[train_master['ThirdParty_Info_Period6_12']>=15]['ThirdParty_Info_Period6_12']=15
train_master[train_master['ThirdParty_Info_Period7_12']>=15]['ThirdParty_Info_Period7_12']=15
train_master[train_master['ThirdParty_Info_Period1_13']>=100000]['ThirdParty_Info_Period1_13']=100000
train_master[train_master['ThirdParty_Info_Period2_13']>=100000]['ThirdParty_Info_Period2_13']=100000
train_master[train_master['ThirdParty_Info_Period3_13']>=100000]['ThirdParty_Info_Period3_13']=100000
train_master[train_master['ThirdParty_Info_Period4_13']>=100000]['ThirdParty_Info_Period4_13']=100000
train_master[train_master['ThirdParty_Info_Period5_13']>=100000]['ThirdParty_Info_Period5_13']=100000
train_master[train_master['ThirdParty_Info_Period6_13']>=100000]['ThirdParty_Info_Period6_13']=100000
train_master[train_master['ThirdParty_Info_Period7_13']>=100000]['ThirdParty_Info_Period7_13']=100000
train_master[train_master['ThirdParty_Info_Period1_14']>=80000]['ThirdParty_Info_Period1_14']=80000
train_master[train_master['ThirdParty_Info_Period2_14']>=80000]['ThirdParty_Info_Period2_14']=80000
train_master[train_master['ThirdParty_Info_Period3_14']>=80000]['ThirdParty_Info_Period3_14']=80000
train_master[train_master['ThirdParty_Info_Period4_14']>=80000]['ThirdParty_Info_Period4_14']=80000
train_master[train_master['ThirdParty_Info_Period5_14']>=80000]['ThirdParty_Info_Period5_14']=80000
train_master[train_master['ThirdParty_Info_Period6_14']>=80000]['ThirdParty_Info_Period6_14']=80000
train_master[train_master['ThirdParty_Info_Period7_14']>=80000]['ThirdParty_Info_Period7_14']=80000
train_master[train_master['ThirdParty_Info_Period1_15']>=20000]['ThirdParty_Info_Period1_15']=20000
train_master[train_master['ThirdParty_Info_Period2_15']>=20000]['ThirdParty_Info_Period2_15']=20000
train_master[train_master['ThirdParty_Info_Period3_15']>=20000]['ThirdParty_Info_Period3_15']=20000
train_master[train_master['ThirdParty_Info_Period4_15']>=20000]['ThirdParty_Info_Period4_15']=20000
train_master[train_master['ThirdParty_Info_Period5_15']>=20000]['ThirdParty_Info_Period5_15']=20000
train_master[train_master['ThirdParty_Info_Period6_15']>=20000]['ThirdParty_Info_Period6_15']=20000
train_master[train_master['ThirdParty_Info_Period7_15']>=20000]['ThirdParty_Info_Period7_15']=20000
train_master[train_master['ThirdParty_Info_Period1_16']>=100000]['ThirdParty_Info_Period1_16']=100000
train_master[train_master['ThirdParty_Info_Period2_16']>=100000]['ThirdParty_Info_Period2_16']=100000
train_master[train_master['ThirdParty_Info_Period3_16']>=100000]['ThirdParty_Info_Period3_16']=100000
train_master[train_master['ThirdParty_Info_Period4_16']>=100000]['ThirdParty_Info_Period4_16']=100000
train_master[train_master['ThirdParty_Info_Period5_16']>=100000]['ThirdParty_Info_Period5_16']=100000
train_master[train_master['ThirdParty_Info_Period6_16']>=100000]['ThirdParty_Info_Period6_16']=100000
train_master[train_master['ThirdParty_Info_Period7_16']>=100000]['ThirdParty_Info_Period7_16']=100000
train_master[train_master['ThirdParty_Info_Period1_17']>=50000]['ThirdParty_Info_Period1_17']=50000
train_master[train_master['ThirdParty_Info_Period2_17']>=50000]['ThirdParty_Info_Period2_17']=50000
train_master[train_master['ThirdParty_Info_Period3_17']>=50000]['ThirdParty_Info_Period3_17']=50000
train_master[train_master['ThirdParty_Info_Period4_17']>=50000]['ThirdParty_Info_Period4_17']=50000
train_master[train_master['ThirdParty_Info_Period5_17']>=50000]['ThirdParty_Info_Period5_17']=50000
train_master[train_master['ThirdParty_Info_Period6_17']>=50000]['ThirdParty_Info_Period6_17']=50000
train_master[train_master['ThirdParty_Info_Period7_17']>=50000]['ThirdParty_Info_Period7_17']=50000    '''
train_master.fillna(-9999,inplace=True)

from sklearn import preprocessing


var_lst = [\
'UserInfo_2',\
'UserInfo_4',\
'UserInfo_5',\
'UserInfo_6',\
'UserInfo_7',\
'UserInfo_8',\
'UserInfo_9',\
'UserInfo_19',\
'UserInfo_20',\
'UserInfo_22',\
'UserInfo_23',\
'UserInfo_24',\
'Education_Info2',\
'Education_Info3',\
'Education_Info4',\
'Education_Info6',\
'Education_Info7',\
'Education_Info8',\
'WeblogInfo_19',\
'WeblogInfo_20',\
'WeblogInfo_21',\
'ListingInfo',\
'ListingInfo_M',\
'UserInfo_2_province',\
'UserInfo_4_province',\
'UserInfo_2_class',\
'UserInfo_4_class'
]
for var in var_lst:
    le = preprocessing.LabelEncoder()
    train_master[var]=le.fit_transform(train_master[var])

In [ ]:
# Data Merge
train_final = train_master.join(train_part1,how='left')
train_final = train_final.join(train_part2,how='left')

drop_lst = [\
'WeblogInfo_49',\
'ListingInfo',\

'UserInfo_2',\
'UserInfo_20',\
'UserInfo_4',\
'UserInfo_8',\
]
train_final.drop(drop_lst,axis=1,inplace=True)

print train_final.shape


In [ ]:
code_lst=[\
'Education_Info2',\
'Education_Info3',\
'Education_Info4',\
'Education_Info6',\
'Education_Info8',\
'ListingInfo_M',\
'SocialNetwork_1',\
'SocialNetwork_11',\
'SocialNetwork_12',\
'SocialNetwork_13',\
'SocialNetwork_14',\
'SocialNetwork_17',\
'SocialNetwork_2',\
'SocialNetwork_7',\
'UserInfo_1',\
'UserInfo_10',\
'UserInfo_11',\
'UserInfo_12',\
'UserInfo_13',\
'UserInfo_14',\
'UserInfo_15',\
'UserInfo_16',\
'UserInfo_19',\
'UserInfo_2_class',\
'UserInfo_2_province',\
'UserInfo_22',\
'UserInfo_23',\
'UserInfo_3',\
'UserInfo_4_class',\
'UserInfo_4_province',\
'UserInfo_5',\
'UserInfo_6',\
'UserInfo_7',\
'UserInfo_9',\
'WeblogInfo_1',\
'WeblogInfo_11',\
'WeblogInfo_12',\
'WeblogInfo_13',\
'WeblogInfo_19',\
'WeblogInfo_2',\
'WeblogInfo_20',\
'WeblogInfo_21',\
'WeblogInfo_23',\
'WeblogInfo_24',\
'WeblogInfo_25',\
'WeblogInfo_26',\
'WeblogInfo_27',\
'WeblogInfo_28',\
'WeblogInfo_29',\
'WeblogInfo_3',\
'WeblogInfo_30',\
'WeblogInfo_31',\
'WeblogInfo_32',\
'WeblogInfo_33',\
'WeblogInfo_34',\
'WeblogInfo_35',\
'WeblogInfo_36',\
'WeblogInfo_37',\
'WeblogInfo_38',\
'WeblogInfo_39',\
'WeblogInfo_40',\
'WeblogInfo_41',\
'WeblogInfo_42',\
'WeblogInfo_43',\
'WeblogInfo_44',\
'WeblogInfo_45',\
'WeblogInfo_46',\
'WeblogInfo_47',\
'WeblogInfo_48',\
'WeblogInfo_50',\
'WeblogInfo_51',\
'WeblogInfo_52',\
'WeblogInfo_53',\
'WeblogInfo_54',\
'WeblogInfo_55',\
'WeblogInfo_56',\
'WeblogInfo_57',\
'WeblogInfo_58',\
]
enc = preprocessing.OneHotEncoder()
train_final_one = train_final.join(pd.DataFrame(enc.fit_transform(train_final[code_lst].replace({-1:9999,-9999:9999})).toarray(),index=train_final.index),how='left')
train_final_one.to_csv(train_path+'train_final_data.csv')



In [ ]:
varlst = [\
'ThirdParty_Info_Period1_5',\
'ThirdParty_Info_Period5_4',\
'ThirdParty_Info_Period3_5',\
'ThirdParty_Info_10_max',\
'ThirdParty_Info_9_sum',\
'ThirdParty_Info_10_mean',\
'ThirdParty_Info_Period2_4',\
'ThirdParty_Info_10_nuz',\
'ThirdParty_Info_Period4_5',\
'ThirdParty_Info_10_std',\
'ThirdParty_Info_Period6_6',\
'ThirdParty_Info_10_sum',\
'ThirdParty_Info_Period1_14',\
'ThirdParty_Info_11_cav',\
'ThirdParty_Info_Period2_12',\
'ThirdParty_Info_11_cmx',\
'ThirdParty_Info_Period3_13',\
'ThirdParty_Info_11_max',\
'ThirdParty_Info_Period4_13',\
'ThirdParty_Info_11_mean',\
'ThirdParty_Info_Period5_13Rank_',\
'ThirdParty_Info_11_min',\
'ThirdParty_Info_Period6_14',\
'ThirdParty_Info_11_std',\
'WeblogInfo_16',\
'ThirdParty_Info_11_sum',\
'ThirdParty_Info_Period1_11Rank_',\
'ThirdParty_Info_12_cav',\
'ThirdParty_Info_Period1_2',\
'ThirdParty_Info_12_max',\
'ThirdParty_Info_Period1_8',\
'ThirdParty_Info_12_mean',\
'ThirdParty_Info_Period2_16',\
'ThirdParty_Info_12_std',\
'ThirdParty_Info_Period2_8',\
'ThirdParty_Info_12_sum',\
'ThirdParty_Info_Period3_17',\
'ThirdParty_Info_13_cav',\
'ThirdParty_Info_Period4_1',\
'ThirdParty_Info_13_cmx',\
'ThirdParty_Info_Period4_17',\
'ThirdParty_Info_13_max',\
'ThirdParty_Info_Period4_9',\
'ThirdParty_Info_13_mean',\
'ThirdParty_Info_Period5_17',\
'ThirdParty_Info_13_min',\
'ThirdParty_Info_Period5_8',\
'ThirdParty_Info_13_std',\
'ThirdParty_Info_Period6_2',\
'ThirdParty_Info_13_sum',\
'UserInfo_18',\
'ThirdParty_Info_14_cav',\
'WeblogInfo_5',\
'ThirdParty_Info_14_cmx',\
'ThirdParty_Info_Period1_10',\
'ThirdParty_Info_14_max',\
'ThirdParty_Info_Period1_13',\
'ThirdParty_Info_14_mean',\
'ThirdParty_Info_Period1_16',\
'ThirdParty_Info_14_min',\
'ThirdParty_Info_Period1_3',\
'ThirdParty_Info_14_std',\
'ThirdParty_Info_Period1_6',\
'ThirdParty_Info_14_sum',\
'ThirdParty_Info_Period2_1',\
'ThirdParty_Info_15_cav',\
'ThirdParty_Info_Period2_14',\
'ThirdParty_Info_15_cmx',\
'ThirdParty_Info_Period2_2',\
'ThirdParty_Info_15_max',\
'ThirdParty_Info_Period2_6',\
'ThirdParty_Info_15_mean',\
'ThirdParty_Info_Period3_11Rank_',\
'ThirdParty_Info_15_min',\
'ThirdParty_Info_Period3_15',\
'ThirdParty_Info_15_std',\
'ThirdParty_Info_Period3_3',\
'ThirdParty_Info_15_sum',\
'ThirdParty_Info_Period3_7',\
'ThirdParty_Info_16_cav',\
'ThirdParty_Info_Period4_11',\
'ThirdParty_Info_16_cmx',\
'ThirdParty_Info_Period4_15',\
'ThirdParty_Info_16_max',\
'ThirdParty_Info_Period4_3',\
'ThirdParty_Info_16_mean',\
'ThirdParty_Info_Period4_7',\
'ThirdParty_Info_16_min',\
'ThirdParty_Info_Period5_11',\
'ThirdParty_Info_16_std',\
'ThirdParty_Info_Period5_15',\
'ThirdParty_Info_16_sum',\
'ThirdParty_Info_Period5_2',\
'ThirdParty_Info_17_cav',\
'ThirdParty_Info_Period5_6',\
'ThirdParty_Info_17_cmx',\
'ThirdParty_Info_Period6_11',\
'ThirdParty_Info_17_max',\
'ThirdParty_Info_Period6_16',\
'ThirdParty_Info_17_mean',\
'ThirdParty_Info_Period6_4',\
'ThirdParty_Info_17_min',\
'ThirdParty_Info_Period6_8',\
'ThirdParty_Info_17_std',\
'UserInfo_4_8_same',\
'ThirdParty_Info_17_sum',\
'WeblogInfo_18',\
'ThirdParty_Info_2_cav',\
'WeblogInfo_7',\
'ThirdParty_Info_2_cmx',\
'ThirdParty_Info_Period1_1',\
'ThirdParty_Info_2_max',\
'ThirdParty_Info_Period1_11',\
'ThirdParty_Info_2_mean',\
'ThirdParty_Info_Period1_12',\
'ThirdParty_Info_2_min',\
'ThirdParty_Info_Period1_13Rank_',\
'ThirdParty_Info_2_msx',\
'ThirdParty_Info_Period1_15',\
'ThirdParty_Info_2_nuz',\
'ThirdParty_Info_Period1_17',\
'ThirdParty_Info_2_std',\
'ThirdParty_Info_Period1_2Rank_',\
'ThirdParty_Info_2_sum',\
'ThirdParty_Info_Period1_4',\
'ThirdParty_Info_3_cav',\
'ThirdParty_Info_Period1_5Rank_',\
'ThirdParty_Info_3_cmx',\
'ThirdParty_Info_Period1_7',\
'ThirdParty_Info_3_max',\
'ThirdParty_Info_Period1_9',\
'ThirdParty_Info_3_mean',\
'ThirdParty_Info_Period2_11',\
'ThirdParty_Info_3_min',\
'ThirdParty_Info_Period2_13',\
'ThirdParty_Info_3_std',\
'ThirdParty_Info_Period2_15',\
'ThirdParty_Info_3_sum',\
'ThirdParty_Info_Period2_17',\
'ThirdParty_Info_4_cav',\
'ThirdParty_Info_Period2_3',\
'ThirdParty_Info_4_cmx',\
'ThirdParty_Info_Period2_5',\
'ThirdParty_Info_4_max',\
'ThirdParty_Info_Period2_7',\
'ThirdParty_Info_4_mean',\
'ThirdParty_Info_Period3_1',\
'ThirdParty_Info_4_min',\
'ThirdParty_Info_Period3_12',\
'ThirdParty_Info_4_std',\
'ThirdParty_Info_Period3_14',\
'ThirdParty_Info_4_sum',\
'ThirdParty_Info_Period3_16',\
'ThirdParty_Info_5_cav',\
'ThirdParty_Info_Period3_2',\
'ThirdParty_Info_5_cmx',\
'ThirdParty_Info_Period3_4',\
'ThirdParty_Info_5_max',\
'ThirdParty_Info_Period3_6',\
'ThirdParty_Info_5_mean',\
'ThirdParty_Info_Period3_8',\
'ThirdParty_Info_5_min',\
'ThirdParty_Info_Period4_10Rank_',\
'ThirdParty_Info_5_std',\
'ThirdParty_Info_Period4_12',\
'ThirdParty_Info_5_sum',\
'ThirdParty_Info_Period4_14',\
'ThirdParty_Info_6_cav',\
'ThirdParty_Info_Period4_16',\
'ThirdParty_Info_6_cmx',\
'ThirdParty_Info_Period4_2',\
'ThirdParty_Info_6_max',\
'ThirdParty_Info_Period4_4',\
'ThirdParty_Info_6_mean',\
'ThirdParty_Info_Period4_6',\
'ThirdParty_Info_6_min',\
'ThirdParty_Info_Period4_8',\
'ThirdParty_Info_6_std',\
'ThirdParty_Info_Period5_1',\
'ThirdParty_Info_6_sum',\
'ThirdParty_Info_Period5_13',\
'ThirdParty_Info_7_cav',\
'ThirdParty_Info_Period5_14',\
'ThirdParty_Info_7_cmx',\
'ThirdParty_Info_Period5_16',\
'ThirdParty_Info_7_max',\
'ThirdParty_Info_Period5_17Rank_',\
'ThirdParty_Info_7_mean',\
'ThirdParty_Info_Period5_3',\
'ThirdParty_Info_7_min',\
'ThirdParty_Info_Period5_5',\
'ThirdParty_Info_7_std',\
'ThirdParty_Info_Period5_7',\
'ThirdParty_Info_7_sum',\
'ThirdParty_Info_Period6_1',\
'ThirdParty_Info_8_cav',\
'ThirdParty_Info_Period6_13',\
'ThirdParty_Info_8_cmx',\
'ThirdParty_Info_Period6_15',\
'ThirdParty_Info_8_max',\
'ThirdParty_Info_Period6_17',\
'ThirdParty_Info_8_mean',\
'ThirdParty_Info_Period6_3',\
'ThirdParty_Info_8_min',\
'ThirdParty_Info_Period6_5',\
'ThirdParty_Info_8_msx',\
'ThirdParty_Info_Period6_7',\
'ThirdParty_Info_8_std',\
'UserInfo_17',\
'ThirdParty_Info_8_sum',\
'UserInfo_2_4_same',\
'ThirdParty_Info_9_cav',\
'WeblogInfo_15',\
'ThirdParty_Info_9_cmx',\
'WeblogInfo_17',\
'ThirdParty_Info_9_max',\
'WeblogInfo_4',\
'ThirdParty_Info_9_mean',\
'WeblogInfo_6',\
'ThirdParty_Info_9_nuz',\
'WeblogInfo_8',\
'ThirdParty_Info_9_std',\
'ThirdParty_Info_10_cav',\
'SocialNetwork_3',\
'Dayslogtolisting',\
'daysfirst_type_act_21_2',\
'7',\
'ratio_cnt_loginfo41_4_listday',\
'20',\
'cnt_weekday0',\
'24',\
'dayslast_RESIDENCEADDRESS',\
'57',\
'price',\
'58',\
'ratio_cnt_loginfo47_4_listday',\
'77',\
'ThirdParty_Info_1_nuz',\
'79',\
'daysfirst_type_act_0_1',\
'86',\
'Daysfirsttolisting',\
'92',\
'dayslast_type_act_2_1',\
'98',\
'Education_Info5',\
'99',\
'ratio_cnt_first_log',\
'100',\
'ratio_cnt_loginfo43_2_listday',\
'101',\
'ratio_cnt_update_MOBILEPHONE_listday',\
'106',\
'ThirdParty_Info_1_cmx',\
'107',\
'cnt_update_listday_3',\
'108',\
'days_since_most_log',\
'112',\
'daysfirst_type_act_2_1',\
'114',\
'daysfirst_type_act_6_-4',\
'126',\
'dayslast_EDUCATIONID',\
'128',\
'dayslast_type_act_0_1',\
'150',\
'dayslast_type_act_6_-4',\
'165',\
'Duration_log',\
'232',\
'less_nation_info4',\
'238',\
'prob_ipo_f30d',\
'240',\
'ratio_cnt_loginfo40_1_listday',\
'253',\
'ratio_cnt_loginfo420_1_listday',\
'258',\
'ratio_cnt_loginfo46_-4_listday',\
'267',\
'ratio_cnt_update_EDUCATIONID_listday',\
'269',\
'ratio_cnt_update_TURNOVER_listday',\
'270',\
'SocialNetwork_9',\
'277',\
'ThirdParty_Info_1_mean',\
'287',\
'ThirdParty_Info_1_sum',\
'288',\
'cnt_update_listday_7',\
'291',\
'days_between_two_log',\
'294',\
'daysfirst_LASTUPDATEDATE',\
'409',\
'daysfirst_type_act_1_4',\
'410',\
'daysfirst_type_act_20_1',\
'454',\
'daysfirst_type_act_3_2',\
'463',\
'Daysfirstlogtolisting',\
'amt_ipo_f15d',\
'dayslast_CITYID',\
'amt_ipo_f30d',\
'dayslast_MOBILEPHONE',\
'amt_ipo_f7d',\
'dayslast_RESIDENCEPHONE',\
'avg_cnt_log',\
'dayslast_type_act_1_4',\
'cnt_ipo_f15d',\
'dayslast_type_act_20_1',\
'cnt_ipo_f30d',\
'Dayslogstd',\
'cnt_ipo_f7d',\
'Daysupdatetolisting',\
'cnt_log',\
'Education_Info1',\
'cnt_log_days_listday_7',\
'Education_Info7',\
'cnt_log_listday',\
'ListingInfo_weekno',\
'cnt_log_listday_15',\
'prob_ipo_f15d',\
'cnt_log_listday_3',\
'prob_ipo_f7d',\
'cnt_log_listday_30',\
'ratio_cnt_first_update',\
'cnt_log_listday_7',\
'ratio_cnt_loginfo41_1_listday',\
'cnt_log_type_actcnt_listday_0',\
'ratio_cnt_loginfo42_1_listday',\
'cnt_log_weekday1',\
'ratio_cnt_loginfo421_2_listday',\
'cnt_log_weekday3',\
'ratio_cnt_loginfo44_1_listday',\
'cnt_log_weekday6',\
'ratio_cnt_loginfo46_99_listday',\
'cnt_log4_days6_-4_listday',\
'ratio_cnt_update_BYUSERID_listday',\
'cnt_loginfo46_-4_listday',\
'ratio_cnt_update_LASTUPDATEDATE_listday',\
'cnt_loginfo46_-4_listday_0',\
'ratio_cnt_update_RESIDENCETYPEID_listday',\
'cnt_loginfo46_-4_listday_15',\
'SocialNetwork_10',\
'cnt_loginfo46_-4_listday_30',\
'SocialNetwork_8',\
'cnt_loginfo46_-4_listday_7',\
'ThirdParty_Info_1_cav',\
'cnt_loginfo47_4_listday_7',\
'ThirdParty_Info_1_max',\
'cnt_type_act_3_2',\
'ThirdParty_Info_1_min',\
'cnt_type_act_6_-4',\
'ThirdParty_Info_1_std',\
'cnt_update',\
'5',\
'ThirdParty_Info_Period3_10',\
'ThirdParty_Info_Period1_1Rank_',\
'ThirdParty_Info_Period3_13Rank_',\
'ThirdParty_Info_Period4_1Rank_',\
'ThirdParty_Info_Period1_14Rank_',\
'ThirdParty_Info_Period5_14Rank_',\
'ThirdParty_Info_Period3_11',\
'ThirdParty_Info_Period5_16Rank_',\
'UserInfo_24',\
'ThirdParty_Info_Period5_9',\
'ThirdParty_Info_Period2_6Rank_',\
'ThirdParty_Info_12_nuz',\
'ratio_cnt_update_RESIDENCEPHONE_listday',\
'260',\
'cnt_log_days_listday_15',\
'daysfirst_type_act_6_107',\
'cnt_first_log',\
'85',\
'daysfirst_type_act_1_1',\
'daysfirst_type_act_7_4',\
'cnt_loginfo47_4_listday_30',\
'12',\
'cnt_update_cnt_listday',\
'302',\
'cnt_weekday2',\
'dayslast_DISTRICTID',\
'daysfirst_QQ',\
'318',\
'daysfirst_type_act_1_3',\
'dayslast_HASBUYCAR',\
'less_nation_info2',\
'dayslast_LASTUPDATEDATE',\
'amt_ipo',\
'319',\
'88',\
'dayslast_QQ',\
'ratio_cnt_loginfo41_2_listday',\
'cnt_log_weekday0',\
'172',\
'402',\
'daysfirst_EDUCATIONID',\
'dayslast_type_act_1_1',\
'daysfirst_MOBILEPHONE',\
'dayslast_type_act_7_4',\
'daysfirst_RESIDENCEADDRESS',\
'cnt_log4_days6_99_listday',\
'daysfirst_type_act_1_2',\
'Daysupdatestd',\
'cnt_log_days_listday',\
'Duration_update',\
'cnt_log_days_listday_30',\
'257',\
'UserInfo_2_4_province_same',\
'ThirdParty_Info_12_cmx',\
'ThirdParty_Info_Period3_15Rank_',\
'ThirdParty_Info_Period2_10',\
'ThirdParty_Info_Period7_15Rank_',\
'ThirdParty_Info_Period3_4Rank_',\
'ThirdParty_Info_Period4_3Rank_',\
'ThirdParty_Info_13_msx',\
'cnt_first_update',\
'dayslast_type_act_0_12',\
'cnt_log_weekday2',\
'cnt_loginfo46_-4_listday_3',\
'dayslast_MARRIAGESTATUSID',\
'280',\
'dayslast_RESIDENCETYPEID',\
'283',\
'174',\
'daysfirst_DISTRICTID',\
'cnt_loginfo43_2_listday',\
'248',\
'dayslast_type_act_1_3',\
'497',\
'147',\
'cnt_log_weekday5',\
'ListingInfo_weekday',\
'ThirdParty_Info_Period1_16Rank_',\
'ThirdParty_Info_Period2_12Rank_',\
'UserInfo_2_20_same',\
'ThirdParty_Info_Period1_9Rank_',\
'ThirdParty_Info_Period4_8Rank_',\
'ThirdParty_Info_Period5_8Rank_',\
'ThirdParty_Info_Period4_9Rank_',\
'ThirdParty_Info_Period4_16Rank_',\
'ThirdParty_Info_Period5_11Rank_',\
'ThirdParty_Info_Period2_9',\
'ThirdParty_Info_Period1_7Rank_',\
'ThirdParty_Info_Period1_4Rank_',\
'ThirdParty_Info_Period3_8Rank_',\
'293',\
'dayslast_type_act_6_107',\
'162',\
'dayslast_type_act_1_2',\
'cnt_update_listday_30',\
'465',\
'daysfirst_PROVINCEID',\
'104',\
'daysfirst_type_act_0_12',\
'428',\
'cnt_log_2moretype_act_listday_0',\
'daysfirst_CITYID',\
'140',\
'cnt_loginfo47_4_listday_15',\
'cnt_log_weekday4',\
'ratio_cnt_update_COMPANYADDRESS_listday',\
'453',\
'ratio_cnt_update_COMPANYPHONE_listday',\
'cnt_update_2moreitem_listday_7',\
'ratio_cnt_update_DISTRICTID_listday',\
'dayslast_RESIDENCEYEARS',\
'dayslast_IDNUMBER',\
'81',\
'ratio_cnt_update_HASBUYCAR_listday',\
'dayslast_type_act_6_4',\
'cnt_LASTUPDATEDATE',\
'ratio_cnt_loginfo420_300_listday',\
'daysfirst_MARRIAGESTATUSID',\
'ratio_cnt_update_RESIDENCEADDRESS_listday',\
'cnt_update_cnt_listday_30',\
'ThirdParty_Info_6_msx',\
'ThirdParty_Info_Period4_11Rank_',\
'ThirdParty_Info_Period6_1Rank_',\
'ThirdParty_Info_Period2_13Rank_',\
'ThirdParty_Info_Period4_13Rank_',\
'ThirdParty_Info_3_msx',\
'ThirdParty_Info_Period3_7Rank_',\
'ThirdParty_Info_Period6_8Rank_',\
'ThirdParty_Info_Period2_3Rank_',\
'ThirdParty_Info_Period3_9',\
'ThirdParty_Info_Period7_6Rank_',\
'ThirdParty_Info_Period2_16Rank_',\
'daysfirst_HASBUYCAR',\
'ratio_cnt_loginfo40_12_listday',\
'daysfirst_RESIDENCEPHONE',\
'8',\
'ratio_cnt_loginfo41_3_listday',\
'181',\
'ratio_cnt_update_QQ_listday',\
'dayslast_PHONE',\
'168',\
'dayslast_REALNAME',\
'cnt_update_listday_15',\
'97',\
'days_between_two_update',\
'dayslast_type_act_3_2',\
'daysfirst_ISCASH',\
'14',\
          'target',\
          'test_ind'
]
train_final_subset = train_final_one[varlst]

In [ ]:
train_df = train_final_subset[(train_final_subset['test_ind']==0)|(train_final_subset['test_ind']==2)]
test_df = train_final_subset[train_final_subset['test_ind']==1]

dtrain  = xgb.DMatrix(train_df.drop(['target','test_ind'],axis=1),train_df.target)
dval  = xgb.DMatrix(test_df.drop(['target','test_ind'],axis=1))
scr_tk = []
for i in range(10):
    print "The %d round start" %(i)
    params = {"objective": "binary:logistic", "eta": 0.02, "max_depth": 5, "min_child_weight": 15,"silent": 1, "subsample": 0.8,"colsample_bytree": 0.8,\
               "eval_metric":"auc",'seed':i*50, 'alpha':0,'lambda':1} 
    num_round = 1650
    bst = xgb.train( params, dtrain, num_round,verbose_eval=100)
    scr0 = bst.predict(dval)   
    params = {"objective": "binary:logistic", "eta": 0.02, "max_depth": 4, "min_child_weight": 10,"silent": 1, "subsample": 0.8,"colsample_bytree": 0.8,\
               "eval_metric":"auc",'seed':i*50, 'alpha':0,'lambda':1} 
    num_round = 2200
    bst = xgb.train( params, dtrain, num_round,verbose_eval=100)
    scr1 = bst.predict(dval)   
    params = {"objective": "binary:logistic", "eta": 0.02, "max_depth": 3, "min_child_weight": 5,"silent": 1, "subsample": 0.8,"colsample_bytree": 0.8,\
               "eval_metric":"auc",'seed':i*50, 'alpha':0,'lambda':1} 
    num_round = 3800 
    bst = xgb.train( params, dtrain, num_round,verbose_eval=100)
    scr2 = bst.predict(dval)
    params = {"objective": "binary:logistic", "eta": 0.02, "max_depth": 6, "min_child_weight":15,"silent": 1, "subsample": 0.8,"colsample_bytree": 0.8,\
               "eval_metric":"auc",'seed':i*50, 'alpha':0,'lambda':1} 
    num_round = 1500
    bst = xgb.train( params, dtrain, num_round,verbose_eval=100)
    scr3 = bst.predict(dval)
    test_df['score'+str(i)] = (scr0+scr1+scr2+scr3)/(4*1.0)
    scr_tk.append([scr0,scr1,scr2,scr3])

scr_test = test_df[['score'+str(i) for i in range(10)]].mean(axis=1)
test_df['score'] = np.round(scr_test,4)
test_df.to_csv(train_path+'test_score_ensemble0417.csv',encoding = 'utf-8', columns=['score'])